In [1]:
from datasets import load_dataset
from transformers import AutoModelForQuestionAnswering, pipeline, AutoTokenizer
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import nltk



dataset = load_dataset("squad")
train = dataset['train']
validation = dataset['validation']

model_checkpoint = "atharvamundada99/bert-large-question-answering-finetuned-legal"
pretrained_model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Found cached dataset parquet (C:/Users/DELL/.cache/huggingface/datasets/parquet/plain_text-57edf78d6033ac9a/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
print(pretrained_model.training)
pretrained_model.eval()
print(pretrained_model.training)

False
False


In [3]:
question_answerer = pipeline("question-answering", model=pretrained_model, tokenizer=tokenizer)

In [4]:
total = 0
correct = 0
EM = 0
BLEU = 0
bblleeuu = 0
errors = []
for record in tqdm(validation):
        # try:
                total += 1
                if (total % 1000 == 0):
                        print(f"Correct:\t\t {correct}, out of {total}: {100*correct/total}%")
                        print(f"EM:\t\t\t {EM}, out of {total}: {100*EM/total}%")
                        print(f"BLEU:\t\t\t {BLEU}, out of {total}: {100*BLEU/total}%")
                        print(f"BLEU Score:\t\t {bblleeuu}, out of {total}: {100*bblleeuu/total}%")
                result = question_answerer(question=record['question'], context=record['context'], truncation=True, padding=True, return_tensors='pt')
                # result = QA(model, tokenizer,record['question'], record['context'])
                n = min(len(result['answer'].split()), 4)
                # n = min(len(result.split()), 4)
                if n == 0:
                        BLEUscore = 0
                else:
                        weights = [1.0/n]*n
                        BLEUscore = nltk.translate.bleu_score.sentence_bleu([record['answers']['text'][0].lower()], result['answer'].lower(), weights=weights)
                        # BLEUscore = nltk.translate.bleu_score.sentence_bleu([record['answers']['text'][0].lower()], result.lower(), weights=weights)
                if result['answer'] != '' and (result['answer'].lower() in record['answers']['text'][0].lower() or record['answers']['text'][0].lower() in result['answer'].lower()):
                # if result != '' and (result.lower() in record['answers']['text'][0].lower() or record['answers']['text'][0].lower() in result.lower()):
                        correct += 1
                if record['answers']['text'][0].lower() == result['answer'].lower():
                # if record['answers']['text'][0].lower() == result.lower():
                        EM += 1
                if BLEUscore > 0.5:
                        BLEU += 1
                bblleeuu += BLEUscore
                 
        # except Exception as e:
        #         errors.append(total)
        #         print(f"Error at {total}: {e}")
        #         continue
print(f"Correct: {correct}, out of {total}: {100*correct/total}%")
print(f"EM: {EM}, out of {total}: {100*EM/total}%")
print(f"BLEU: {BLEU}, out of {total}: {100*BLEU/total}%")
print(f"BLEU Score: {bblleeuu}, out of {total}: {100*bblleeuu/total}%")



  1%|          | 114/10570 [00:34<26:31,  6.57it/s] c:\Users\DELL\anaconda3\Lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\Users\DELL\anaconda3\Lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
  4%|▍         | 439/10570 [04:16<2:09:14,  1.31it/s]c:\Users\DELL\anaconda3\Lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram over

Correct:		 942, out of 1000: 94.2%
EM:			 762, out of 1000: 76.2%
BLEU:			 878, out of 1000: 87.8%
BLEU Score:		 872.7723680467068, out of 1000: 87.27723680467068%


 19%|█▉        | 1999/10570 [21:14<1:42:44,  1.39it/s]

Correct:		 1869, out of 2000: 93.45%
EM:			 1432, out of 2000: 71.6%
BLEU:			 1703, out of 2000: 85.15%
BLEU Score:		 1697.4421047529827, out of 2000: 84.87210523764912%


 28%|██▊       | 2999/10570 [32:45<1:26:53,  1.45it/s]

...

Correct:		 8362, out of 9000: 92.91111111111111%
EM:			 5986, out of 9000: 66.5111111111111%
BLEU:			 7270, out of 9000: 80.77777777777777%
BLEU Score:		 7292.617317963844, out of 9000: 81.02908131070937%


 95%|█████████▍| 9999/10570 [2:09:58<06:42,  1.42it/s]

Correct:		 9279, out of 10000: 92.79%
EM:			 6578, out of 10000: 65.78%
BLEU:			 8042, out of 10000: 80.42%
BLEU Score:		 8068.880321520323, out of 10000: 80.68880321520324%


100%|██████████| 10570/10570 [2:17:41<00:00,  1.28it/s]

Correct: 9789, out of 10570: 92.61116367076632%
EM: 6901, out of 10570: 65.28855250709556%
BLEU: 8477, out of 10570: 80.19867549668874%
BLEU Score: 8501.452133048304, out of 10570: 80.43001071947307%
